In [18]:
# import os
# import sys
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import pandas as pd
from quantfreedom.utils.caching import clear_cache
from quantfreedom.backtester.base.base import plot_trades_all_info, rsi_above_entries
from quantfreedom.backtester.nb.simulate import simulate_from_signals
from quantfreedom.backtester.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
pd.options.display.float_format = '{:,.2f}'.format
# clear_cache()

# qf.CCXTData.data_download(
#     exchange='bybit',
#     start='2021-11-10T00:00:00Z',
#     end='2022-01-23T00:00:00Z',
#     symbol='BTCUSDT',
#     timeframe='30m',
# ).to_csv('30minshort.csv')

prices = pd.read_csv('30minshort.csv', index_col='open_time')
entries = rsi_above_entries(
    timeperiods=[20],
    above_ranges=[50],
    prices=prices.close
)


In [19]:
df_array, order_records, df_cart_array = simulate_from_signals(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=entries.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.ShortEntry,
    max_equity_risk_pct=3.,
    risk_rewards=[3,5],
    size_pct=1.,
    sl_pcts=[4, 5],
    # tsl_true_or_false=True,
    # tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    # tsl_trail_by_pct=1,
    # tsl_when_pct_from_avg_entry=[3,4],
)


In [20]:
df_or = pd.DataFrame(order_records)
for i in range(len(OrderType._fields)):
    df_or.replace({'order_type': {i: OrderType._fields[i]}}, inplace=True)
df_or.head(10)
    

,order_id,or_set,ind_set,bar,size_value,price,avg_entry,fees_paid,order_type,real_pnl,equity,sl_prices,tsl_prices,tp_prices
0,0,0,0,27,242.58,"66,420.50","66,420.50",NaN,ShortEntry,NaN,"1,000.00","69,077.32",NaN,"58,131.41"
1,1,0,0,28,242.58,"68,028.50","67,214.88",NaN,ShortEntry,NaN,"1,000.00","69,903.48",NaN,"58,826.66"
2,2,0,0,29,242.58,"68,391.00","67,602.40",NaN,ShortEntry,NaN,"1,000.00","70,306.50",NaN,"59,165.82"
3,3,0,0,308,727.73,"59,165.82","67,602.40",0.82,ShortTP,90.00,"1,090.00","70,306.50",NaN,"59,165.82"
4,4,0,0,357,264.41,"59,169.50","59,169.50",NaN,ShortEntry,NaN,"1,090.00","61,536.28",NaN,"51,785.32"
5,5,0,0,358,264.41,"60,518.00","59,836.15",NaN,ShortEntry,NaN,"1,090.00","62,229.60",NaN,"52,368.77"
6,6,0,0,359,264.41,"60,572.00","60,079.44",NaN,ShortEntry,NaN,"1,090.00","62,482.62",NaN,"52,581.70"
7,7,0,0,1144,793.23,"52,581.70","60,079.44",0.89,ShortTP,98.10,"1,188.10","62,482.62",NaN,"52,581.70"
8,8,0,0,1190,288.21,"48,826.00","48,826.00",NaN,ShortEntry,NaN,"1,188.10","50,779.04",NaN,"42,732.66"
9,9,0,0,1191,288.21,"49,214.50","49,019.48",NaN,ShortEntry,NaN,"1,188.10","50,980.26",NaN,"42,901.99"


In [21]:
pd.DataFrame(df_array).sort_values(by='to_the_upside', ascending=False)


,or_set,ind_set,win_rate,gains_pct,to_the_upside,total_trades,total_BE,total_fees,total_pnl,ending_eq
0,0,0,80.00,36.92,0.86,5.00,0,4.74,369.23,"1,369.23"
2,2,0,50.00,54.84,0.80,8.00,0,8.36,548.38,"1,548.38"
1,1,0,66.67,32.82,0.78,6.00,0,4.46,328.16,"1,328.16"
3,3,0,66.67,28.28,0.57,3.00,0,2.12,282.82,"1,282.83"


In [22]:
df_cart = pd.DataFrame(df_cart_array).dropna(axis='columns', thresh=1)
for i in range(len(SL_BE_or_Trail_BasedOn._fields)):
    df_cart.replace({'tsl_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
    df_cart.replace({'sl_to_be_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
df_cart.T

,0,1,2,3
order_settings_id,0.00,1.00,2.00,3.00
max_equity_risk_pct,3.00,3.00,3.00,3.00
risk_rewards,3.00,3.00,5.00,5.00
size_pct,1.00,1.00,1.00,1.00
sl_pcts,4.00,5.00,4.00,5.00


In [23]:
plot_trades_all_info(
    open_prices=prices.open,
    high_prices=prices.high,
    low_prices=prices.low,
    close_prices=prices.close,
    order_records=order_records[0:20],
)


In [24]:
clear_cache()